In [9]:
# =========================================================
# FINAL MODEL + FASTAPI 
# =========================================================

from mlflow.tracking import MlflowClient
import mlflow
import pandas as pd
import joblib
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split

from custom_transformers import Log1pTransformer
from pydantic import BaseModel

# ---------------------------------------------------------
# 0. Load modeling dataset and create X_train / y_train
# ---------------------------------------------------------
df = pd.read_parquet("modeling_dataset_clean.parquet")
print("✅ Loaded modeling_dataset_clean.parquet")
print("Columns:", df.columns.tolist())


binary_cols = [c for c in df.columns if df[c].dropna().isin([0, 1]).all()]
if len(binary_cols) == 1:
    target_col = binary_cols[0]
else:
    target_col = df.columns[-1]

print(f"✅ Detected target column: {target_col}")

X = df.drop(target_col, axis=1)
y = df[target_col]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
print("✅ X_train shape:", X_train.shape)
print("✅ y_train shape:", y_train.shape)

# ---------------------------------------------------------
# 1. Connect to MLflow and find best run
# ---------------------------------------------------------
mlflow.set_tracking_uri("https://dagshub.com/menna1996/avoidable_ed_ml_project.mlflow")
client = MlflowClient()

experiment = client.get_experiment_by_name("Final_Project_Experiments")
runs = client.search_runs(experiment_ids=[experiment.experiment_id])

records = []
for run in runs:
    f1 = run.data.metrics.get("test_f1")
    if f1 is not None:
        records.append({
            "run_id": run.info.run_id,
            "run_name": run.data.tags.get("mlflow.runName", run.info.run_id),
            "test_f1": f1
        })

df_runs = pd.DataFrame(records).sort_values("test_f1", ascending=False)
best_run = df_runs.iloc[0]

best_run_id = best_run["run_id"]
best_run_name = best_run["run_name"]

print("✅ Best run:", best_run_name)

# ---------------------------------------------------------
# 2. Extract hyperparameters
# ---------------------------------------------------------
run_data = client.get_run(best_run_id)
raw_params = run_data.data.params

clean_params = {}
for k, v in raw_params.items():
    if k.startswith("clf__"):
        k = k.replace("clf__", "")
    if k in ["n_estimators", "max_depth", "min_samples_split", "min_samples_leaf"]:
        clean_params[k] = int(v)
    elif v in ["True", "False"]:
        clean_params[k] = (v == "True")
    else:
        clean_params[k] = v

print("✅ Cleaned params:", clean_params)

# ---------------------------------------------------------
# 3. EXACT feature lists from your experiment
# ---------------------------------------------------------
numeric_features = [
    "AGE_AT_END_REF_YR",
    "total_paid_amt",
    "primary_dx_chronic_flag",
    "bodysystem_respiratory",
    "bodysystem_circulatory",
    "bodysystem_infectious",
    "bodysystem_digestive",
    "bodysystem_mentalbehavioral",
    "bodysystem_musculoskeletal",
    "bodysystem_neoplasms",
    "bodysystem_nervoussystem",
    "bodysystem_injurypoisoning",
    "bodysystem_skin",
    "bodysystem_genitourinary",
    "bodysystem_endocrine",
    "bodysystem_bloodimmune",
    "bodysystem_symptoms",
    "bodysystem_externalcauses",
    "bodysystem_congenital",
    "bodysystem_perinatal",
    "bodysystem_pregnancy",
    "bodysystem_dental",
    "bodysystem_eye",
    "bodysystem_ear",
    "bodysystem_healthstatus",
    "bodysystem_unacceptable",
]

categorical_features = [
    "SEX_IDENT_CD",
    "BENE_RACE_CD",
    "YEAR"
]

missing_in_data = [c for c in numeric_features + categorical_features if c not in X_train.columns]
if missing_in_data:
    print("ERROR: Missing features:", missing_in_data)
    raise ValueError("Feature mismatch between experiment and modeling_dataset_clean.parquet")

# ---------------------------------------------------------
# 4. Rebuild preprocessing pipeline
# ---------------------------------------------------------
numeric_transformer = Pipeline(steps=[
    ("log1p", Log1pTransformer()),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

# ---------------------------------------------------------
# 5. Rebuild final model
# ---------------------------------------------------------
rf_final = RandomForestClassifier(
    class_weight="balanced",
    n_jobs=-1,
    random_state=42,
    **clean_params,
)

final_pipeline = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("clf", rf_final),
])

# ---------------------------------------------------------
# 6. Fit final model
# ---------------------------------------------------------
final_pipeline.fit(X_train, y_train)
print("✅ Final model fitted.")

# ---------------------------------------------------------
# 7. Save final model
# ---------------------------------------------------------
joblib.dump(final_pipeline, "final_avoidable_ed_model.joblib")
print("✅ Saved final_avoidable_ed_model.joblib")

# ---------------------------------------------------------
# 8. Extract input features for FastAPI
# ---------------------------------------------------------
input_features = preprocessor.feature_names_in_
print("✅ Input features:", list(input_features))

# ---------------------------------------------------------
# 9. Generate FastAPI schema
# ---------------------------------------------------------
fields = "\n".join([f"    {feat}: float" for feat in input_features])

schema_text = f"""
class EDVisitInput(BaseModel):
{fields}
"""

print(schema_text)

# ---------------------------------------------------------
# 10. Generate main.py 
# ---------------------------------------------------------
main_py = f"""
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import joblib
import pandas as pd

model = joblib.load("final_avoidable_ed_model.joblib")

class EDVisitInput(BaseModel):
{fields}

app = FastAPI()

@app.get("/")
def root():
    return {{"message": "Avoidable ED Prediction API"}}

@app.post("/predict")
def predict(data: EDVisitInput):
    try:
        df = pd.DataFrame([data.dict()])
        print("Received input:", df.to_dict(orient="records")[0])

        df.columns = df.columns.str.strip()

        categorical_cols = ["SEX_IDENT_CD", "BENE_RACE_CD", "YEAR"]
        for col in categorical_cols:
            if col in df.columns:
                df[col] = df[col].astype(str)

        df.fillna("missing", inplace=True)

        pred = model.predict(df)[0]
        prob = model.predict_proba(df)[0][1]

        print("Prediction:", pred, "Probability:", prob)
        return {{"prediction": int(pred), "probability": float(prob)}}

    except Exception as e:
        print("Prediction error:", str(e))
        raise HTTPException(status_code=500, detail="Prediction failed")
"""

with open("main.py", "w") as f:
    f.write(main_py)

print("✅ main.py generated.")


✅ Loaded modeling_dataset_clean.parquet
Columns: ['AGE_AT_END_REF_YR', 'SEX_IDENT_CD', 'BENE_RACE_CD', 'YEAR', 'total_paid_amt', 'primary_dx_chronic_flag', 'bodysystem_respiratory', 'bodysystem_circulatory', 'bodysystem_infectious', 'bodysystem_digestive', 'bodysystem_mentalbehavioral', 'bodysystem_musculoskeletal', 'bodysystem_neoplasms', 'bodysystem_nervoussystem', 'bodysystem_injurypoisoning', 'bodysystem_skin', 'bodysystem_genitourinary', 'bodysystem_endocrine', 'bodysystem_bloodimmune', 'bodysystem_symptoms', 'bodysystem_externalcauses', 'bodysystem_congenital', 'bodysystem_perinatal', 'bodysystem_pregnancy', 'bodysystem_dental', 'bodysystem_eye', 'bodysystem_ear', 'bodysystem_healthstatus', 'bodysystem_unacceptable', 'Avoidable_ED_Visit']
✅ Detected target column: Avoidable_ED_Visit
✅ X_train shape: (6047, 29)
✅ y_train shape: (6047,)
✅ Best run: Exp2_RandomForest
✅ Cleaned params: {'max_depth': 20, 'min_samples_split': 5, 'n_estimators': 100}
✅ Final model fitted.
✅ Saved final_

In [ ]:
# Validation 

!pip install uvicorn[standard]

from custom_transformers import Log1pTransformer
import joblib
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# -------------------------------
# Load dataset
# -------------------------------
df = pd.read_parquet("modeling_dataset_clean.parquet")
print("✅ Loaded dataset")

# Detect target column (binary or last column)
binary_cols = [c for c in df.columns if df[c].dropna().isin([0,1]).all()]
target_col = binary_cols[0] if len(binary_cols)==1 else df.columns[-1]
print("✅ Target column:", target_col)

X = df.drop(target_col, axis=1)
y = df[target_col]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# -------------------------------
# Load model
# -------------------------------
model = joblib.load("final_avoidable_ed_model.joblib")
print("✅ Model loaded")

# -------------------------------
# Check feature alignment
# -------------------------------
expected = list(model.named_steps["preprocess"].feature_names_in_)
missing = [c for c in expected if c not in X_train.columns]
extra = [c for c in X_train.columns if c not in expected]

print("✅ Expected features:", expected)
print("❌ Missing:", missing) if missing else print("✅ No missing features")
print("⚠️ Extra:", extra) if extra else print("✅ No extra features")

# -------------------------------
# Prediction tests
# -------------------------------
sample = X_train.iloc[[0]]
pred = model.predict(sample)[0]
prob = model.predict_proba(sample)[0][1]
print("✅ Single prediction OK:", pred, prob)

preds = model.predict(X_train.head(50))
probs = model.predict_proba(X_train.head(50))[:,1]
print("✅ Batch prediction OK")

# -------------------------------
# NaN check
# -------------------------------
if np.isnan(preds).any() or np.isnan(probs).any():
    print("❌ NaNs found")
else:
    print("✅ No NaNs in predictions")

print("\n🎉 FINAL CHECK COMPLETE — Model is deployment‑ready 🎉")

✅ Loaded dataset
✅ Target column: Avoidable_ED_Visit
✅ Model loaded
✅ Expected features: ['AGE_AT_END_REF_YR', 'SEX_IDENT_CD', 'BENE_RACE_CD', 'YEAR', 'total_paid_amt', 'primary_dx_chronic_flag', 'bodysystem_respiratory', 'bodysystem_circulatory', 'bodysystem_infectious', 'bodysystem_digestive', 'bodysystem_mentalbehavioral', 'bodysystem_musculoskeletal', 'bodysystem_neoplasms', 'bodysystem_nervoussystem', 'bodysystem_injurypoisoning', 'bodysystem_skin', 'bodysystem_genitourinary', 'bodysystem_endocrine', 'bodysystem_bloodimmune', 'bodysystem_symptoms', 'bodysystem_externalcauses', 'bodysystem_congenital', 'bodysystem_perinatal', 'bodysystem_pregnancy', 'bodysystem_dental', 'bodysystem_eye', 'bodysystem_ear', 'bodysystem_healthstatus', 'bodysystem_unacceptable']
✅ No missing features
✅ No extra features
✅ Single prediction OK: 0 0.038439883318979254
✅ Batch prediction OK
✅ No NaNs in predictions

🎉 FINAL CHECK COMPLETE — Model is deployment‑ready 🎉


In [13]:
!pip install -U jupyter-book
